In [1]:
from libs import generation as gn
from libs import models as md
from libs import settings as st
from libs import utils as ut

import numpy as np

import sqlalchemy as sa
import pandas as pd
import datetime as dt

### Create table

In [ ]:
conn = st.conn_test

md.Base.metadata.create_all(conn)

### Data insertion

### Genre base

### Director base

### Title base

### Language base

### Date base

### Movies_Genre base

### Movies_Director base

### Raking_Movies base